In [1]:
import pandas as pd
import Bio.PDB
import numpy as np
import os
import tqdm

In [13]:
#read all results
resdf = pd.read_table('structaln_score.tsv' , header = None)
print(resdf)

                    0               1     2      3             4   5    6   \
0       A0A060LXZ4.pdb  A0A060LXZ4.pdb  2487  1.000  5.344000e-43   0  361   
1       A0A060LXZ4.pdb  A0A060M1P9.pdb  1418  0.526  2.304000e-26   0  361   
2       A0A060LXZ4.pdb  A0A060M707.pdb  1128  0.398  1.259000e-21   1  361   
3       A0A060LXZ4.pdb  A0A060LST4.pdb  1076  0.468  6.192000e-23   2  361   
4       A0A060LXZ4.pdb  A0A060LTA2.pdb   942  0.318  1.327000e-18   1  359   
...                ...             ...   ...    ...           ...  ..  ...   
647746      V5MM16.pdb  A0A2Z6BN59.pdb    32  0.102  2.244000e+00  35  312   
647747      V5MM16.pdb  A0A7H0Y2E7.pdb    31  0.131  1.201000e+00  92  376   
647748      V5MM16.pdb      Q9A103.pdb    30  0.109  1.404000e+00  27  383   
647749      V5MM16.pdb  A0A4U9YQS3.pdb    29  0.112  1.579000e+00  27  399   
647750      V5MM16.pdb  A0A3S4SG61.pdb    22  0.120  3.586000e+00  27  366   

         7   8    9    10                                      

In [15]:
#map hits to each struc
hits = {}
#proportion of structures that need to map to a residue for it to be kep in the final struct
hitthresh = .8
strcutfold = './structs/'
with tqdm.tqdm(total=len(resdf[0].unique())) as pbar:
    for i,q in enumerate(resdf[0].unique()):
        sub = resdf[resdf[0] == q]
        hitvec = np.zeros((1,sub.iloc[0][7]))
        for idx,r in sub.iterrows():
            hitvec[0,r[5]:r[6]] = hitvec[0,r[5]:r[6]]+1
        hitvec /= len(resdf[0].unique())
        core = np.where(hitvec>hitthresh)[1]
        hits[q]= (np.amin(core), np.amax(core))
        pbar.set_description('processed: %d' % (1 + i))
        pbar.update(1)  

processed: 805: 100%|█████████████████████████| 805/805 [01:58<00:00,  6.79it/s]


In [16]:
print(hits)

{'A0A060LXZ4.pdb': (15, 344), 'A0A075R948.pdb': (78, 386), 'A0A0F7FGL9.pdb': (62, 415), 'A0A0M3RAQ7.pdb': (10, 322), 'A0A109WF09.pdb': (17, 315), 'A0A1B1LDW3.pdb': (13, 317), 'A0A1E1G9T9.pdb': (4, 282), 'A0A1U9K4D6.pdb': (74, 369), 'A0A1W6AJ49.pdb': (24, 332), 'A0A1Y0IMR9.pdb': (73, 343), 'A0A1Y0IWF6.pdb': (73, 345), 'A0A223CZV5.pdb': (72, 345), 'A0A285BFV3.pdb': (9, 363), 'A0A2I5JZA3.pdb': (12, 328), 'A0A2K9IZF0.pdb': (61, 356), 'A0A2Z6BN59.pdb': (42, 257), 'A0A385T342.pdb': (78, 387), 'A0A3Q9HRH2.pdb': (15, 272), 'A0A3S9TJ74.pdb': (34, 328), 'A0A4P7A0I1.pdb': (62, 350), 'A0A518V219.pdb': (114, 447), 'A0A5J6SU43.pdb': (48, 328), 'A0A6G7JEB2.pdb': (19, 315), 'A0A6I6I7J9.pdb': (20, 277), 'A0A6I6UPL9.pdb': (63, 346), 'A0A6I7FCS7.pdb': (27, 320), 'A0A7G1IQM5.pdb': (5, 277), 'A0A7H0Y2E7.pdb': (15, 261), 'A0A7T5ESM5.pdb': (25, 355), 'A0A7Z2FZP5.pdb': (11, 314), 'A0A826GVE8.pdb': (21, 326), 'A7GVL1.pdb': (22, 296), 'A9VV35.pdb': (22, 315), 'B7IZ99.pdb': (21, 324), 'C2MNK1.pdb': (10, 297), 'C

In [17]:

corefolder = 'core_structs/'
structfolder = 'structs/'
#make core struct folder
try:
    os.mkdir(corefolder)
except:
    print('already')
#parse each struct and output core to folder 
parser = Bio.PDB.PDBParser()
with tqdm.tqdm(total=len(hits)) as pbar:
    for i,q in enumerate(hits):
        struct = parser.get_structure(q.split('.')[0] , structfolder+q )
        #zero based indexing...
        struct_core = Bio.PDB.Dice.extract(struct ,'A' , hits[q][0]+1 , hits[q][1]+1 , corefolder+q  )
        pbar.set_description('processed: %d' % (1 + i))
        pbar.update(1)            #todo ngl view some structs vs core
#recalc tree w foldtree outside the notebook

already


processed: 805: 100%|█████████████████████████| 805/805 [01:02<00:00, 12.96it/s]


In [18]:
#redo tree
import subprocess, shlex
import sys
sys.path.append('../../src/')
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import foldseek2tree
db = foldseek2tree.runFoldseekdb(corefolder , corefolder )
print(db)

core_structs/structblobDB exists and will be overwritten
createdb core_structs/ core_structs/structblobDB 

MMseqs Version:  	1.3c64211
Chain name mode  	0
Write lookup file	1
Threads          	4
Verbosity        	3

Output file: core_structs/structblobDB
[=================================================================] 823 0s 889ms
Time for merging to structblobDB_ss: 0h 0m 0s 2ms
Time for merging to structblobDB_h: 0h 0m 0s 1ms
Time for merging to structblobDB_ca: 0h 0m 0s 10ms
Time for merging to structblobDB: 0h 0m 0s 1ms
Ignore 0 out of 823.
Too short: 0, incorrect  0.
Time for processing: 0h 0m 0s 927ms
core_structs/structblobDB 


In [20]:
treefile = foldseek2tree.structblob2tree(corefolder , 'logs/' )

logs/structblobDB exists and will be overwritten
createdb core_structs/ logs/structblobDB 

MMseqs Version:  	1.3c64211
Chain name mode  	0
Write lookup file	1
Threads          	4
Verbosity        	3

Output file: logs/structblobDB
[=================================================================] 823 0s 936ms
Time for merging to structblobDB_ss: 0h 0m 0s 1ms
Time for merging to structblobDB_h: 0h 0m 0s 1ms
Time for merging to structblobDB_ca: 0h 0m 0s 10ms
Time for merging to structblobDB: 0h 0m 0s 1ms
Ignore 0 out of 823.
Too short: 0, incorrect  0.
Time for processing: 0h 0m 0s 981ms
search logs/structblobDB logs/structblobDB logs/aln tmp -a --max-seqs 3000 

MMseqs Version:              	1.3c64211
Seq. id. threshold           	0
Coverage threshold           	0
Coverage mode                	0
Max reject                   	2147483647
Max accept                   	2147483647
Add backtrace                	true
TMscore threshold            	0.5
TMalign fast                 	1
Preload m

In [22]:
#postprocess and madroot
pp = foldseek2tree.postprocess(treefile)
